In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
x = torch.empty(5, 4)
print(x)

In [ ]:
torch.ones(3, 3)

In [ ]:
torch.zeros(2)

In [ ]:
torch.rand(5, 6)

In [ ]:
l = [13, 4]
r = np.array([4, 56, 7])
torch.tensor(l)

In [ ]:
torch.tensor(r)

In [ ]:
x.size()[0]

In [ ]:
type(x)

In [ ]:
x = torch.rand(2, 2)
y = torch.rand(2, 2)
x + y

In [ ]:
x

In [ ]:
y

In [ ]:
x+y

In [ ]:
torch.add(x, y)

In [ ]:
y.add(x)

In [ ]:
y.add_(x)

In [ ]:
y

In [ ]:
x = torch.rand(8, 8)
print(x.size())

In [ ]:
a = x.view(64)
print(a.size())

In [ ]:
b =  x.view(-1, 4, 4)
print(b.size())

In [ ]:
x = torch.rand(8,8)
y = x.numpy()
type(y)

In [ ]:
x = torch.ones(1)
print(x.item())

In [ ]:
x = torch.FloatTensor(range(5)).unsqueeze(1)
y = 2*x + torch.rand(5,1)  # y는 실제 값으로 임의로 5개를 만들어 준다.
num_features = x.shape[1]  # 변수의 개수를 저장하는 num_features. x의 크기는 torch.Size([5,1])이므로 인스턴스의 개수가 5개이고 변수의 개수가 1개인 데이터

In [ ]:
# 역전파 설정 requires_grad=True
w = torch.randn(num_features, 1, requires_grad=True)  # 선형식은 y = wx + b로 표현.
b = torch.randn(1, requires_grad=True)  # 편향 b는 모든 인스턴스에 동일한 숫자를 더해주는 것이므로 크기가 1인 텐서로 정의

In [ ]:
# 경사하강법
learning_rate = 1e-3  # 학습률
optimizer = torch.optim.SGD([w,b], lr=learning_rate)

loss_stack = []
for epoch in range(100):
    optimizer.zero_grad()  # 매 epoch마다 누적된 값을 optimizer.zero_grad()를 통해 초기화
    y_hat = torch.matmul(x,w) + b  # 회귀식 모델을 이용하여 예측값을 산출
    loss = torch.mean((y_hat - y) ** 2)  # 예측값과 실제값을 이용하여 손실함수를 계산. 사용된 함수는 MSE
    loss.backward()  # 역전파의 기준을 손실함수로 정의
    optimizer.step()  # 미리 정의한 optimizer를 이용하여 최적화를 시행
    loss_stack.append(loss.item())  # 그래프를 그리기 위해 손실함수 값만 loss_stack에 넣는다. item()을 사용하지 않으면 loss텐서 전체를 저장하게 된다.

    if epoch % 100 == 0:  # epoch이 100으로 나눠 떨어질 때마다 손실함수 값을 출력
        print(f'Epoch {epoch} : {loss.item()}')

In [ ]:
# 최종 학습된 w, b로 예측값을 산출
with torch.no_grad():  # 최적화를 사용하지 않으므로 requires_grad를 비활성화
    y_hat = torch.matmul(x,w) + b

In [ ]:
# 손실함수와 회귀선 그래프

plt.figure(figsize=(10, 5))
plt.subplot(121)
plt.plot(loss_stack)
plt.title("Loss")
plt.subplot(122)
plt.plot(x, y, '.b')
plt.plot(x, y_hat, 'r-')
plt.legend(['ground truth','prediction'])
plt.title("Prediction")
plt.show()

In [ ]:
import torchvision  # 이미지와 관련된 파이토치 라이브러리
import torchvision.transforms as tr  # 이미지 전처리 기능들을 제공하는 라이브러리
from torch.utils.data import DataLoader, Dataset  # 데이터를 모델에 사용할 수 있도록 정리해 주는 라이브러리

In [ ]:
# tr.Compose 내에 원하는 전처리를 차례대로 넣어주면 된다.
# 예시에서는 16x16으로 이미지 크기 변환 후 텐서 타입으로 변환한다.
# 만약 원본 이미지의 너비, 높이가 다를 경우 너비, 높이를 각각 지정을 해야 하기 때문에
# tr.Resize((16,16))이라고 입력해야 한다. 
transf = tr.Compose([tr.Resize(16), tr.ToTensor()])

In [ ]:
# torchvison.datasets에서 제공하는 CIFAR10 데이터를 불러온다 (CIFAR10은 클래스가 10개인 이미지 데이터 셋)
# root에는 다운로드받을 경로를 입력한다. train=True이면 학습 데이터를 불러오고 False면 테스트 데이터
# 마지막으로 미리 선언한 전처리를 사용하기 위해 transform=transf을 입력한다.
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transf)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transf)

In [ ]:
# 현재 이미지 사이즈는 3 x 16 x 16 (채널수 x 너비 x 높이)
# 3은 채널 수, 16 x 16은 이미지의 너비와 높이 (위에 transf로 리사이즈한 숫자가 적용)
# 일반적인 컬러 사진은 RGB 이미지이기 때문에 채널이 3개
print(trainset[0][0].size())

In [ ]:
# DataLoader는 데이터를 미니 배치 형태로 만들어 준다. 따라서 배치 데이터에 관한 배치 사이즈 및 셔플 여부 등을 선택할 수 있다.
# 즉, batch_size=50, shuffle=True은 무작위로 데이터를 섞어 한 번에 50개의 이미지를 묶은 배치로 제공하겠다는 의미
trainloader = DataLoader(trainset, batch_size=50, shuffle=True)
testloader = DataLoader(testset, batch_size=50, shuffle=False)

In [ ]:
# CIFAR10의 학습 이미지 수
len(trainset)

In [ ]:
# 배치 개수
len(trainloader)

In [ ]:
# trainloader의 첫 번째 배치 불러오기
images, labels = next(iter(trainloader))
print(images.size())  # (배치크기 x 채널수 x 너비 x 높이)

In [ ]:
# image[1]의 크기는 (3, 16, 16)이다.
# 이때 그림을 그려주기 위해서, 채널 수가 가장 뒤로 인 (16, 16, 3) 형태로 만들어야 한다.
# permute(1,2,0)은 기존 차원의 위치인 0,1,2를 1,2,0으로 바꿔주는 함수
# 따라서 0번째의 크기가 3인 텐서를 마지막으로 보내고, numpy()를 이용해 넘파이 배열로 변환
oneshot = images[1].permute(1,2,0).numpy()

In [ ]:
# 그림 그리기
plt.figure(figsize=(2,2))
plt.imshow(oneshot)
plt.axis("off")
plt.show()

In [ ]:
# 같은 클래스 별로 폴더를 정리한 경우
# 데이터가 같은 클래스 별로 미리 폴더를 정리한 경우, ImageFolder 하나로 개인 데이터를 사용할 수 있다.
# 또한 별도의 라벨링이 필요 없으며 폴더 별로 자동으로 라벨링을 한다.
# 예를 들어 class폴더에 tiger, lion 폴더(./class/tiger와 ./class/lion)를 미리 만든다.
# 마지막으로 ImageForlder에 상위 폴더 ./class를 입력하면 이미지와 라벨이 정리되어 데이터를 불러온다.
transf = tr.Compose([tr.Resize(128), tr.ToTensor()])
trainset = torchvision.datasets.ImageFolder(root='./class', transform=transf)
trainloader = DataLoader(trainset, batch_size=1, shuffle=False)

In [ ]:
# 정리되지 않은 커스텀 데이터 불러오기

# from torch.utils.data import Dataset

# class test_class(Dataset):  # Dataset을 상속받아 DataLoader에서 배치 단위로 불러올 수 있게 해준다.
#     def __init__(self):  # 데이터 세팅에 필요한 것들을 미리 정의하는 역할
#         .....
#     def __getitime__(self, index):  # DataLoader를 통해 샘플이 요청되면, 인덱스에 해당하는 샘플을 찾아준다.
#         .....
#     def __len__(self):  # 크기를 반환
#         .....

In [ ]:
# 커스텀 데이터 세트 예시

train_images = np.random.randint(256, size=(100, 32, 32, 3))/255
train_labels = np.random.randint(2, size=(100, 1))

class TensorData(Dataset):
    def __init__(self, x_data, y_data):  # x,y데이터를 tensor로 변환
        self.x_data = torch.FloatTensor(x_data)
        self.x_data = self.x_data.permute(0,3,1,2)  # 그림을 그리기 위해, 배열 순서 변경. (이미지수x너비x높이x채널수) -> (이미지수x채널수x너비x높이)
        self.y_data = torch.LongTensor(y_data)
        self.len = self.y_data.shape[0]  # 입력 데이터의 개수에 대한 변수

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]
    
    def __len__(self):
        return self.len

In [ ]:
train_data = TensorData(train_images, train_labels)
train_loader = DataLoader(train_data, batch_size=10, shuffle=True)

In [ ]:
# 커스텀 데이터와 커스텀 전처리 사용하기
# 파이토치 전처리 함수들은 이미지의 경우 PIL-Image타입이거나 Teonsor타입일 때 사용 가능
# 제공하지 않는 기능에 대해서는 직접 구현이 필요

# 텐서 변환 전처리 클래스 정의
class ToTensor:  # 입력 데이터를 텐서 데이터로 변환해 주고 학습에 맞는 크기로 변환하는 작업을 담당
    def __call__(self, sample):  # 전처리는 MyDataset클래스의 sample을 불러와 작업하기 때문에 __call__함수를 이용
        inputs, labels = sample
        inputs = torch.FloatTensor(inputs)
        inputs = inputs.permute(2,0,1)  # __call__ 함수는 입력값을 하나씩 불러오기 때문에 permute(0,3,1,2)가 아닌 permute(2,0,1)
        # return inputs, labels # labels이 이미 텐서 형식이거나, 다른 전처리 없이 바로 사용할 수 있을 때 적합
        return inputs, torch.LongTensor(labels)  # 분류 문제에서는 레이블이 LongTensor 형식으로 있어야 CrossEntropyLoss 같은 손실 함수와 호환되므로, 레이블이 정수형 텐서로 필요할 때 유용

In [ ]:
# CutOut 전처리 클래스 정의
# 이미지 내부에 무작위로 사각형 영역을 선택하여 0으로 만드는 데이터 증식 방법

class CutOut:
    # CutOut 발생 비율을 받기 위해 __init__ 함수를 사용하여 ratio를 받는다.
    # 기본 ratio는 0.5로 세팅하면 불러온 이미지에 대해서 50% 확률로 CutOut 발현
    def __init__(self, ratio=.5):
        self.ratio = int(1/ratio)
    
    def __call__(self, sample):
        inputs, labels = sample
        active = int(np.random.randint(0, self.ratio, 1))  # 정수뽑기. 50%일 경우 0과 1 중 하나를 뽑게 되고,

        if active == 0:  # 0이면 CutOut를 별현, 1이면 그대로
            _, w, h = inputs.size()  # 이미지의 (채널수), 너비, 높이 받기
            min_len = min(w, h)  # 최소값 구하기
            box_size = int(min_len//4)  # CutOut의 크기를 길이의 최소값 25%로 설정
            idx = int(np.random.randint(0, min_len-box_size, 1))  # CutOut 박스의 좌측 상단 꼭지점 위치를 지정
            inputs[:, idx:idx+box_size, idx:idx+box_size] = 0  # 해당 정사각형 영역의 값을 0으로 대체 (코드를 변형하여 다른 모양으로 변경 가능)
        
        return inputs, labels

In [ ]:
# MyDataset에 전처리를 추가

class MyDataset(Dataset):

    def __init__(self, x_data, y_data, transform=None):  # transform=None 아무것도 적지 않으면 전처리를 사용하지 않겠다
        self.x_data = x_data
        self.y_data = y_data
        self.transform = transform
        self.len = len(y_data)
        self.tensor = ToTensor()

    def __getitem__(self, index):
        # sample = self.x_data[index], self.y_data[index]
        x = self.x_data[index]  # 입력 데이터
        y = self.y_data[index]  # 레이블
        sample = (x, y)

        if self.transform:  # transform이 None이 아니라면 __getitem__에서 sample을 반환하기 전에 전처리를 진행
            # sample = self.transform(sample)
            sample = self.transform(sample)
        else:  # transform = None일 경우, 텐서 변환은 기본적으로 하도록
            # sample = self.tensor(sample)
            x = torch.FloatTensor(x).permute(2, 0, 1)
            sample = (x, y)
        
        return sample
    
    def __len__(self):
        return self.len

In [ ]:
# 파이토치에서 제공하는 텐서 변환 사용이 아닌, 직접 정의한 ToTensor()를 사용
# CutOut은 괄호에 아무 값도 없으므로 발현 비율의 기본값인 0.5로 셋팅
trans = tr.Compose([ToTensor(), CutOut()])
dataset1 = MyDataset(train_images, train_labels, transform=trans)
train_loader1 = DataLoader(dataset1, batch_size=10, shuffle=True)

In [ ]:
import torchvision  # 그리드를 만들어주는 torchvision.utils.make_grid를 사용하기 위해
images1, labels1 = next(iter(train_loader1))

def imshow(img):
    plt.figure(figsize=(10,100))
    plt.imshow(img.permute(1,2,0).numpy())
    plt.show()

imshow(torchvision.utils.make_grid(images1, nrow=10))

In [ ]:
# 커스텀 데이터와 파이토치 제공 전처리 사용하기
# 기능은 있는데 데이터 타입이 다른 경우 PIL-Image 타입으로 변환하여 제공된 전처리를 사용할 수 있다.

class MyTransform:
    def __call__(self, sample):
        inputs, labels = sample
        inputs = torch.FloatTensor(inputs)
        inputs = inputs.permute(2,0,1)
        labels = torch.FloatTensor(labels)

        # tr.Compose는 차례대로 전처리 작업을 하므로 
        # 가장 첫 번째에 tr.PILImage()를 넣어 이미지 타입을 바꿔줄 수 있다.
        transf = tr.Compose([tr.ToPILImage(), tr.Resize(128), tr.ToTensor()])
        final_output = transf(inputs)  # 다음 불러온 샘플을 전처리 작업에 넣어준다

        return final_output, labels

In [ ]:
# 전처리에 정의한 MyTransform()을 넣어준다.
dataset2 = MyDataset(train_images, train_labels, transform=MyTransform())
train_loader2 = DataLoader(dataset2, batch_size=10, shuffle=True)

In [ ]:
# 컨스텀 전처리와 파이토이에서 제공하는 전처리 함께 사용하기
# 위에서 사용한 CutOut과 달리, 라벨은 받지 않고 이미지를 받아 처리하도록 세팅한다.
# 그 이유는 Compose 내부에 있는 제공된 전처리는 이미지만 받아서 처리하기 때문에 그 양식을 맞춰 주어야 한다.
# 위에서 정의한 CutOut은 텐서나 넘파이 배열 타입 모두 작동을 하도록 만들었지만,
# PILImage 타입에서는 오류가 난다. 따라서 tr.ToTensor() 뒤에 CutOut을 배치한다.

# 데이터 증강의 일환
# 이미지의 임의의 부분을 가려 모델이 특정 위치에 의존하지 않도록 학습시키는 데 도움을 준다
class CutOut:
    def __init__(self, ratio=.5):
        self.ratio = int(1/ratio)
    
    def __call__(self, inputs):
        active = int(np.random.randint(0, self.ratio, 1))

        if active == 0:
            _, w, h = inputs.size()
            min_len = min(w, h)  # w, h 중 더 작은 값으로, 이미지의 너비와 높이 중 작은 값
            box_size = int(min_len//4)  # min_len 이미지의 1/4 크기의 작은 정사각형 영역
            idx = int(np.random.randint(0, min_len - box_size, 1))  # 마스킹할 정사각형의 시작 위치를 렌덤하게 지정
            # inputs[채널, 너비(의 일부영역), 높이(의 일부영역)] = 0
            inputs[:, idx:idx+box_size, idx:idx+ box_size] = 0  # 0으로 설정하여 검은색으로 마스킹
        
        return inputs
    
transf = tr.Compose([tr.Resize(128), tr.ToTensor(), CutOut()])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transf)

trainloader = DataLoader(trainset, batch_size=10, shuffle=True)
images, labels = next(iter(trainloader))
imshow(torchvision.utils.make_grid(images, nrow=10))
print(images.size())  # 배치 및 이미지 크기 확인
